In [1]:
import json
import gzip
from io import BytesIO
import boto3
import random
import pandas as pd


In [6]:
S3_BUCKET = "ppa-embeddings-bucket"
S3_INPUT_PREFIX = "target_embeddings/"        # where your original batches are
S3_OUTPUT_PREFIX = "filtered_embeddings/" 

s3 = boto3.client("s3", region_name="us-west-2")


In [13]:
keyword_db = pd.read_csv('Data/ppa_corpus_2025-02-03_1308/ppa_keyword_db.csv')

In [17]:
keyword_db[keyword_db['spelling'].apply(len) > 1]

,page_id,work_id,order,form,tags,counts,contexts,page_text,spelling,source_id,...,author,pub_year,publisher,pub_place,collections,work_type,source,source_url,sort_title,subtitle
0,A01224.36,A01224,36,Elegy,['book'],1,"[""ns chaleur, sans poux, d'amoreuse langueur\n...","\nverse, one Adoniū be added thereunto, as\n\...",['elegiac'],A01224,...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Thomas Orwin,At London,"['Linguistic', 'Literary']",full-work,EEBO-TCP,http://name.umdl.umich.edu/A01224.0001.001,Arcadian rhetorike: or The præcepts of rhetori...,"Greeke, Latin, English, Italian, French, Spani..."
1,A01224.52,A01224,52,Sonnet,['book'],1,"['ebat,\nImperium solenne socer.\n\nSir P. Syd...",Salust. 4. Semaine.\nReglant ensemblement nos ...,['sonet'],A01224,...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Thomas Orwin,At London,"['Linguistic', 'Literary']",full-work,EEBO-TCP,http://name.umdl.umich.edu/A01224.0001.001,Arcadian rhetorike: or The præcepts of rhetori...,"Greeke, Latin, English, Italian, French, Spani..."
2,A01224.53,A01224,53,Sonnet,['book'],1,"['ebat,\nImperium solenne socer.\n\nSir P. Syd...","And,\n—Socer arua Latinus habebat,\nImperium s...",['sonet'],A01224,...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Thomas Orwin,At London,"['Linguistic', 'Literary']",full-work,EEBO-TCP,http://name.umdl.umich.edu/A01224.0001.001,Arcadian rhetorike: or The præcepts of rhetori...,"Greeke, Latin, English, Italian, French, Spani..."
3,A01224.57,A01224,57,Pastoral,['book'],1,"['nnot perswade?\n3, \nBut namelesse hee, for ...",\nword is changed in signification by changing...,['pastoral'],A01224,...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Thomas Orwin,At London,"['Linguistic', 'Literary']",full-work,EEBO-TCP,http://name.umdl.umich.edu/A01224.0001.001,Arcadian rhetorike: or The præcepts of rhetori...,"Greeke, Latin, English, Italian, French, Spani..."
4,A01224.61,A01224,61,Sestina,['book'],2,"['es: but let them passe, and come we to such ...","\nintangled verses: but let them passe, and co...",['sestine'],A01224,...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Thomas Orwin,At London,"['Linguistic', 'Literary']",full-work,EEBO-TCP,http://name.umdl.umich.edu/A01224.0001.001,Arcadian rhetorike: or The præcepts of rhetori...,"Greeke, Latin, English, Italian, French, Spani..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208703,yale.39002088447587.00000052,yale.39002088447587,52,Ode,NaN,2,"['gulph-of destruc-\n"" tion.""\nIt is conceived...","XVI INTRODUCTION.\n"" For.I-will^arise against-...",['ode'],yale.39002088447587,...,"Clarke, George Somers",1810.0,Richard Taylor,London,['Literary'],full-work,HathiTrust,https://hdl.handle.net/2027/yale.39002088447587,Hebrew criticism and poetry : or the patriarch...,with appendixes of readings and interpretation...
208704,yale.39002088447587.00000071,yale.39002088447587,71,Pastoral,NaN,1,['rthens is rather\nunderstood to signify lite...,UPON HIS TWELVE SONS. 15\nZebulon's peopled po...,['pastoral'],yale.39002088447587,...,"Clarke, George Somers",1810.0,Richard Taylor,London,['Literary'],full-work,HathiTrust,https://hdl.handle.net/2027/yale.39002088447587,Hebrew criticism and poetry : or the patriarch...,with appendixes of readings and interpretation...
208705,yale.39002088447587.00000450,yale.39002088447587,450,Hymn,NaN,1,['rb in the future relative. What in the com-\...,394 THE PROPHECY\nAs years approach thy wondro...,['hymn'],yale.39002088447587,...,"Clarke, George Somers",1810.0,Richard Taylor,London,['Literary'],full-work,HathiTrust,https://hdl.handle.net/2027/yale.39002088447587,Hebrew criticism and poetry : or the patriarch...,with appendixes of readings and interpretation...
208706,yul.11891363_000_00.00000049,yul.11891363_000_00,49,Hymn,NaN,1,"['chine,—no, net, 345 6 712345\t12 3\nnor, mov...","﻿PRONUNCIATION.\n45\nuhr for ur in current, be...",['hymn'],yul.11891363_000_00,...,"Josselyn, Calib Bates",1866.0,"Walker, Fuller, & Co.",Boston,['Linguistic'],full-w

In [20]:
pages = (
    keyword_db.groupby("page_id", sort=False)
      .agg({
          "page_text": "first",
          "spelling": lambda x: [item for sublist in x for item in (sublist if isinstance(sublist, list) else [sublist])],
          "work_id": "first",
          "title": "first",
          "author": "first",
          "pub_year": "first"
      })
      .reset_index()
)

# only keep spellings in targets
pages["targets"] = [
    [{"spelling": s} for s in spellings]
    for spellings in pages["spelling"]
]

In [21]:
pages

,page_id,page_text,spelling,work_id,title,author,pub_year,targets
0,A01224.36,"\nverse, one Adoniū be added thereunto, as\n\...",[['elegiac']],A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,[{'spelling': '['elegiac']'}]
1,A01224.52,Salust. 4. Semaine.\nReglant ensemblement nos ...,[['sonet']],A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,[{'spelling': '['sonet']'}]
2,A01224.53,"And,\n—Socer arua Latinus habebat,\nImperium s...",[['sonet']],A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,[{'spelling': '['sonet']'}]
3,A01224.57,\nword is changed in signification by changing...,[['pastoral']],A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,[{'spelling': '['pastoral']'}]
4,A01224.61,"\nintangled verses: but let them passe, and co...",[['sestine']],A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,[{'spelling': '['sestine']'}]
...,...,...,...,...,...,...,...,...
174223,yale.39002088447587.00000052,"XVI INTRODUCTION.\n"" For.I-will^arise against-...",[['ode']],yale.39002088447587,Hebrew criticism and poetry : or the patriarch...,"Clarke, George Somers",1810.0,[{'spelling': '['ode']'}]
174224,yale.39002088447587.00000071,UPON HIS TWELVE SONS. 15\nZebulon's peopled po...,[['pastoral']],yale.39002088447587,Hebrew criticism and poetry : or the patriarch...,"Clarke, George Somers",1810.0,[{'spelling': '['pastoral']'}]
174225,yale.39002088447587.00000450,394 THE PROPHECY\nAs years approach thy wondro...,[['hymn']],yale.39002088447587,Hebrew criticism and poetry : or the patriarch...,"Clarke, George Somers",1810.0,[{'spelling': '['hymn']'}]
174226,yul.11891363_000_00.00000049,"﻿PRONUNCIATION.\n45\nuhr for ur in current, be...",[['hymn']],yul.11891363_000_00,Elements of pronunciation,"Josselyn, Calib Bates",1866.0,[{'spelling': '['hymn']'}]


In [4]:
target_words = ["love", "war", "freedom"]  # example, fill in your actual list
target_set = set(target_words) 

In [24]:
s3 = boto3.client("s3")

key = f"{S3_INPUT_PREFIX}batch_00000.jsonl.gz"  # e.g. f"{S3_INPUT_PREFIX}/usage_batch_00010.jsonl.gz"

# Download only the first few KB of the file
obj = s3.get_object(Bucket=S3_BUCKET, Key=key)
body = obj["Body"].read(1024 * 64)  # read 64 KB max

# Decompress in memory
obj = s3.get_object(Bucket=S3_BUCKET, Key=key)
with gzip.GzipFile(fileobj=BytesIO(obj["Body"].read()), mode="r") as gz:
    # Parse JSON lines
    records = [json.loads(line) for line in gz]

# Load into DataFrame
df = pd.DataFrame(records)


In [25]:
df

,page_id,work_id,title,author,pub_year,form,spelling,char_start,char_end,context,embedding
0,A01224.52,A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Sonnet,sonet,1047,1054,"bebat,\nImperium solenne socer.\n\nSir P. Syd....","[-6.913546562194824, -7.551021575927734, -0.10..."
1,A01224.53,A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Sonnet,sonet,127,134,"bebat,\nImperium solenne socer.\n\nSir P. Syd....","[-8.797904968261719, -13.467477798461914, 0.80..."
2,A01224.57,A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Pastoral,pastoral,1171,1181,"annot perswade?\n3, \nBut namelesse hee, for b...","[-23.346973419189453, -15.783506393432617, -47..."
3,A01224.61,A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Sestina,sestine,114,123,"ses: but let them passe, and come we to such a...","[-61.318145751953125, 81.1879653930664, 5.5105..."
4,A01224.92,A01224,The Arcadian rhetorike: or The præcepts of rhe...,"Fraunce, Abraham, fl. 1587-1633",1588.0,Sonnet,sonet,779,786,"o.\nAnd if I better sing, let me thy Cosma kis...","[-15.393964767456055, 5.531899452209473, -1.20..."
...,...,...,...,...,...,...,...,...,...,...,...
135,A28403.41,A28403,The comparison of Pindar and Horace,"Blondel, François, 1618-1686",1696.0,Ode,odes,541,546,"ce by this Scantling the smiling Gaieties, the...","[6.500802040100098, -14.537841796875, 2.554525..."
136,A28403.41,A28403,The comparison of Pindar and Horace,"Blondel, François, 1618-1686",1696.0,Ode,odes,709,714,o lay by tha• Majestick Severity which appears...,"[10.251133918762207, -5.022252559661865, -4.77..."
137,A28403.43,A28403,The comparison of Pindar and Horace,"Blondel, François, 1618-1686",1696.0,Ode,ode,926,930,Pindar drew upon himself the Iealousie and Ave...,"[11.086874008178711, -19.18686294555664, -26.3..."
138,A28403.47,A28403,The comparison of Pindar and Horace,"Blondel, François, 1618-1686",1696.0,Ode,ode,457,462,ir impetuous Rage\nCan ever in o'erwhelming Fl...,"[-47.49261474609375, 15.214469909667969, -15.0..."
